# Using siibra-python to characterize motor and language areas in the Julich-Brain cytoarchitectonic atlas 


### 1. Import the package 

We start by importing the `siibra` package.

In [ ]:
!pip install siibra siibra-jugex matplotlib nilearn

In [ ]:
import siibra
assert siibra.__version__ >= "0.3a17"

### 2. Select two regions from Julich-Brain

In [ ]:
# access the human atlas
atlas = siibra.atlases.MULTILEVEL_HUMAN_ATLAS

# access Julich-Brain in version 2.9 
jubrain = atlas.get_parcellation('julich 2.9')

# Get two region objects from Julich-Brain
regions = [
    jubrain.decode_region(spec) 
    for spec in ['4p left', '44 left']
]

In [ ]:
# fetch and display their probability maps
from nilearn import plotting
for region in regions:
    pmap = region.get_regional_map(space='mni152', maptype='continuous')
    plotting.plot_stat_map(pmap.fetch(), title=region.name)

### 3. Authenticate to EBRAINS for data retrieval

For accessing features, we have to authenticate to the EBRAINS knowledge graph with a token. By registering to EBRAINS on https://ebrains.eu/register, we can just call `siibra.fetch_ebrains_token()` which will prompt for usernames and password. In this tutorial, we choose a different way and pass a given access token directly:

In [ ]:
siibra.set_ebrains_token(input('Your token:'))

### 4. Access GABAB receptor distributions

In [ ]:
receptor = 'GABAB'

In [ ]:
import matplotlib.pyplot as plt
%matplotlib notebook

fig, axs = plt.subplots(2, len(regions), sharex='row', sharey='row')

for i, region in enumerate(regions):
    
    # retrieve the receptor feature
    feature = siibra.get_features(
        region, siibra.modalities.ReceptorDistribution
    )[-1]
    
    # plot sample autoradiograph for selected receptor
    axs[0, i].imshow(feature.autoradiographs[receptor])
    axs[0, i].axis('off')
    axs[0, i].set_title(region.name)
    
    # Plot the cortical profile of selected receptor
    axs[1, i].plot(
        feature.profiles[receptor].densities.values()
    )
    axs[1, i].set_xlabel('Cortical Depth')
    axs[1, i].grid(True)
    if i==0:
        axs[1, i].set_ylabel(f'{receptor} density {feature.profiles[receptor].unit}')
        

    

### 5. Run a differential gene expression analysis for the two regions

In [ ]:
from siibra_jugex import DifferentialGeneExpression

In [ ]:
# setup and run the analysis
jugex = DifferentialGeneExpression(jubrain)
jugex.add_candidate_genes(['GABBR1', 'GABBR2'])
threshold=0.2
jugex.define_roi1(regions[0].name, maptype=siibra.MapType.CONTINUOUS, threshold=threshold)
jugex.define_roi2(regions[1].name, maptype=siibra.MapType.CONTINUOUS, threshold=threshold)
result = jugex.run(permutations=1000)
print(result['p-values'])

In [ ]:
# plot the extracted sample positions of the microarray data
from nilearn import plotting
for region in regions:
    samples = jugex.get_samples(region.name)
    pmap = region.get_regional_map(
        siibra.spaces.MNI152_2009C_NONL_ASYM, 
        siibra.MapType.CONTINUOUS)    
    display = plotting.plot_roi(pmap.fetch(), cmap="jet", title=region.name)
    display.add_markers([s['mnicoord'] for s in samples.values()])